<a href="https://colab.research.google.com/github/MonkeyWrenchGang/PythonBootcamp/blob/main/day_5/5_2_Pandas_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 5 - Pandas Review:

## Opioids and Medicare Part D


---

## Background

The opioid crisis, stemming from the misuse and addiction to opioid drugs, intersects with Medicare Part D, the prescription drug benefit program under Medicare.

Using our nc_opioid_analysis dataset let's review basic pandas operations

0. importing a CSV file
1. selecting specific columns
2. filtering rows
  - masking
  - queries
3. aggregate()
4. group_by() and aggregate()
5. export to csv and excel





In [ ]:
# supress stupid warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 0. Import Data

```python
opioids = pd.read_csv("https://raw.githubusercontent.com/MonkeyWrenchGang/PythonBootcamp/main/day_4/data/NC_OPIOID_ANALYSIS.csv")
opioids.head()
```

In [ ]:
opioids = pd.read_csv("https://raw.githubusercontent.com/MonkeyWrenchGang/PythonBootcamp/main/day_4/data/NC_OPIOID_ANALYSIS.csv")
opioids.head()

,NPI,Credentials,Gender,Entity_Type,Street_Address_1,Street_Address_2,State_Code,Provider_Type,Num_of_Services,Num_of_Medicare_Beneficiaries,...,HYDROCODONE_script_count,tramadol_script_count,Percocet_script_count,oxycodone_script_count,fentanyl_script_count,levothyroxine_script_count,potassium_script_count,metformin_script_count,diltiazem_script_count,gabapentin_script_count
0,1003007964,PA-C,F,I,3101 LATROBE DR,NaN,NC,Physician Assistant,103.0,82,...,34,0,0,0,0,0,0,0,0,0
1,1003011578,M.D.,F,I,415 N CENTER ST,SUITE 300,NC,Gastroenterology,1281.0,530,...,0,0,0,0,0,0,0,0,0,0
2,1003013756,D.O.,M,I,2651 MORGANTON BLVD SW,NaN,NC,Family Practice,1476.0,474,...,40,32,15,0,0,81,0,173,0,60
3,1003026550,PA-C,M,I,301 YADKIN ST,NaN,NC,Physician Assistant,1176.0,256,...,123,11,309,633,33,0,0,0,0,23
4,1003031691,M.D.,M,I,2800 BLUE RIDGE RD STE 201,NaN,NC,Internal Medicine,1741.0,559,...,0,0,0,0,0,0,0,0,0,0


# 1. Select Specific Columns


---

## Create Results that include:
- phys_city = Full_Name, Credentials, Provider_Type, city_name
  ```python
  phys_city = opioids[["Full_Name", "Credentials", "Provider_Type", "city_name"]]
phys_city.head()
  ```
- phys_billing = Full_Name, Credentials, Provider_Type,Total_Medicare_Payment_Amt, Num_of_Medicare_Beneficiaries

  ```python
  phys_billing = opioids[["Full_Name",
                        "Credentials",
                        "Provider_Type",
                        "Total_Medicare_Payment_Amt",
                        "Num_of_Medicare_Beneficiaries"]]
phys_billing.head()  

  ```

- phys_schooling = Full_Name, Credentials, Provider_Type, Medical_school_name, Graduation_year

In [ ]:
phys_city = opioids[["Full_Name", "Credentials", "Provider_Type", "city_name"]]
phys_city.head()

,Full_Name,Credentials,Provider_Type,city_name
0,OLGA M KRAMER,PA-C,Physician Assistant,Charlotte
1,GAA O RICHARDSON,M.D.,Gastroenterology,Hickory
2,ANDY PHUNG,D.O.,Family Practice,Lenoir
3,LARRY N ROEDIGER,PA-C,Physician Assistant,Albemarle
4,WILLIAM B HALL,M.D.,Internal Medicine,Raleigh


In [ ]:
phys_billing = opioids[["Full_Name",
                        "Credentials",
                        "Provider_Type",
                        "Total_Medicare_Payment_Amt",
                        "Num_of_Medicare_Beneficiaries"]]
phys_billing.head()

,Full_Name,Credentials,Provider_Type,Total_Medicare_Payment_Amt,Num_of_Medicare_Beneficiaries
0,OLGA M KRAMER,PA-C,Physician Assistant,5336.88,82
1,GAA O RICHARDSON,M.D.,Gastroenterology,108433.33,530
2,ANDY PHUNG,D.O.,Family Practice,62902.88,474
3,LARRY N ROEDIGER,PA-C,Physician Assistant,48436.22,256
4,WILLIAM B HALL,M.D.,Internal Medicine,109101.62,559


In [ ]:
phys_schooling = opioids[["Full_Name",
                          "Credentials",
                          "Provider_Type",
                          "Medical_school_name",
                          "Graduation_year"]]
phys_schooling.head()

,Full_Name,Credentials,Provider_Type,Medical_school_name,Graduation_year
0,OLGA M KRAMER,PA-C,Physician Assistant,OTHER,1984.0
1,GAA O RICHARDSON,M.D.,Gastroenterology,UNIVERSITY OF ALABAMA SCHOOL OF MEDICINE,2002.0
2,ANDY PHUNG,D.O.,Family Practice,OTHER,2007.0
3,LARRY N ROEDIGER,PA-C,Physician Assistant,OTHER,1977.0
4,WILLIAM B HALL,M.D.,Internal Medicine,UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL SC...,2004.0


# 2. Filter Rows

## 2.1 Use masking to filter for
- create charlotte_phys by filtering phys_city where city_name == "Charlotte"
  ```python
  mask = phys_city["city_name"] == "Charlotte"
charlotte_phys = phys_city[mask]
charlotte_phys

  ```
- create fivehundredk_pac_billers by filtering **phys_billing** for
"Provider_Type" == "Family Practice" AND "Total_Medicare_Payment_Amt" > 500000
  ```python
  mask = (phys_billing["Provider_Type"] == "Family Practice") & (phys_billing["Total_Medicare_Payment_Amt"] > 500000)
fivehundredk_pac_billers = phys_billing[mask]
fivehundredk_pac_billers
  ```

- create wfu_phys by filtering phys_schooling where Medical_school_name == "BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST UNIVERSITY"
  ```python
  mask = (phys_billing["Provider_Type"] == "Family Practice") & (phys_billing["Total_Medicare_Payment_Amt"] > 500000)
wfu_phys = phys_billing[mask]
wfu_phys
  ```


In [ ]:
mask = phys_city["city_name"] == "Charlotte"
charlotte_phys = phys_city[mask]
charlotte_phys

,Full_Name,Credentials,Provider_Type,city_name
0,OLGA M KRAMER,PA-C,Physician Assistant,Charlotte
16,MELISSA R FLOOD,NP-C,Nurse Practitioner,Charlotte
33,COLEMAN E ALTMAN,D.O.,Dermatology,Charlotte
41,ANTHONY L ASHER,MD,Neurosurgery,Charlotte
50,JOHN M FEDOR,M.D.,Cardiology,Charlotte
...,...,...,...,...
13805,NAGAJYOTHI GAVINI,M.D,Hematology/Oncology,Charlotte
13809,IRENE E EDWARDS,MD,Dermatology,Charlotte
13812,CRAIG D KOLASCH,M.D.,General Surgery,Charlotte
13824,VISHWA S RAJ,M.D.,Physical Medicine and Rehabilitation,Charlotte


In [ ]:
mask = (phys_billing["Provider_Type"] == "Family Practice") & (phys_billing["Total_Medicare_Payment_Amt"] > 500000)
wfu_phys = phys_billing[mask]
wfu_phys

,Full_Name,Credentials,Provider_Type,Total_Medicare_Payment_Amt,Num_of_Medicare_Beneficiaries
2198,TOM A CHRISTENSEN,MD,Family Practice,504984.65,2090
5562,PEGGY S BARNHILL,MD,Family Practice,1013862.98,1075
5639,ELIZABETH P FRY,NaN,Family Practice,532768.85,990
7896,DAVID A MAJURE,MD,Family Practice,510168.42,1287
8724,TED CLONTZ,M.D.,Family Practice,551435.73,1295


In [ ]:
mask = phys_schooling["Medical_school_name"] == "BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST UNIVERSITY"
phys_schooling[mask]

,Full_Name,Credentials,Provider_Type,Medical_school_name,Graduation_year
119,GRAHAM H ERLACHER,M.D.,Emergency Medicine,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1992.0
141,RONALD R CALDWELL,M.D.,Internal Medicine,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1978.0
188,GARY P CRAM,M.D.,Neurosurgery,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1994.0
207,STEPHEN P ROSSER,MD,General Surgery,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1980.0
224,SUZONNE I STRATTON,P.A.-C,Physician Assistant,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1992.0
...,...,...,...,...,...
13778,HAROLD R HOWE,MD,Cardiac Surgery,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1980.0
13783,EDWARD G HILL,M.D.,Neurology,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1980.0
13787,MELISSA R CHAMPE - SEAGLE,M.D.,Internal Medicine,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1995.0
13795,GEORGE C CLINARD,M.D.,Cardiology,BOWMAN GRAY SCHOOL OF MEDICINE OF WAKE FOREST ...,1990.0


## 2.2 Query()

1. create a dataframe "asheville_pac" by querying **phys_city** where Credentials == "PA-C" and city_name == "Asheville"

2. create a dataframe high_averge_patient by quering phys_billing where Total_Medicare_Payment_Amt divided by Num_of_Medicare_Beneficiaries is > 5000
  ```
  phys_billing.query('Total_Medicare_Payment_Amt/Num_of_Medicare_Beneficiaries > 5000')
  ```
3. create old_duke by quering phys_schooling where Medical_school_name == DUKE UNIVERSITY SCHOOL OF MEDICINE and graduation year is before 1980
  ```python
phys_schooling.query('Medical_school_name == "DUKE UNIVERSITY SCHOOL OF MEDICINE" and Graduation_year< 1980')
  ```

In [ ]:
phys_billing.query('Total_Medicare_Payment_Amt/Num_of_Medicare_Beneficiaries > 5000')

,Full_Name,Credentials,Provider_Type,Total_Medicare_Payment_Amt,Num_of_Medicare_Beneficiaries
575,JOHN S SALMON,MD,Hematology,1968153.86,210
1553,PAUL S KIM,M.D.,Medical Oncology,1014968.49,188
1794,JAMES N ANTEZANA,MD,General Surgery,1285538.89,214
5998,CHARLES C THOMAS,MD,Radiation Oncology,1572474.24,282
6485,J. S ROBERTS,MD,Radiation Oncology,1157347.91,139
8388,KEVIN J KERLIN,"M.D., F.A.C.R.O.",Radiation Oncology,1196046.93,204
9408,YATWAH LEUNG,M.D.,Radiation Oncology,1554789.04,274


In [ ]:
phys_schooling.query('Medical_school_name == "DUKE UNIVERSITY SCHOOL OF MEDICINE" and Graduation_year< 1980')


,Full_Name,Credentials,Provider_Type,Medical_school_name,Graduation_year
50,JOHN M FEDOR,M.D.,Cardiology,DUKE UNIVERSITY SCHOOL OF MEDICINE,1976.0
94,EDWARD L TREADWELL,M.D.,Rheumatology,DUKE UNIVERSITY SCHOOL OF MEDICINE,1975.0
311,LISA KAUFMANN,MD,Internal Medicine,DUKE UNIVERSITY SCHOOL OF MEDICINE,1978.0
435,DEANNA C JONES,NaN,Family Practice,DUKE UNIVERSITY SCHOOL OF MEDICINE,1979.0
585,WILLIAM R BERRY,M.D.,Hematology/Oncology,DUKE UNIVERSITY SCHOOL OF MEDICINE,1974.0
...,...,...,...,...,...
13165,JAMES W HATHORN,MD,Hematology/Oncology,DUKE UNIVERSITY SCHOOL OF MEDICINE,1979.0
13223,MICHAEL B SHIPLEY,M.D.,Internal Medicine,DUKE UNIVERSITY SCHOOL OF MEDICINE,1974.0
13340,ROBERT L BLOOMFIELD,MDMS,Internal Medicine,DUKE UNIVERSITY SCHOOL OF MEDICINE,1977.0
13364,JOEL RAPCHIK,MD,Internal Medicine,DUKE UNIVERSITY SCHOOL OF MEDICINE,1971.0


# Rank, Nlargest, NSmallest

1. create a column billing_rank on **phys_billing** then filter where rank is <= 10
2. using **opioids** select top 10 `oxycodone_day_count`, subset to show just  "Full_Name", "Credentials", "Provider_Type",  "Total_Medicare_Payment_Amt", and  "oxycodone_day_count"
3. using **phys_schooling** select the oldest phsycians based on graduation_year


In [ ]:
opioids.columns

Index(['NPI', 'Credentials', 'Gender', 'Entity_Type', 'Street_Address_1',
       'Street_Address_2', 'State_Code', 'Provider_Type', 'Num_of_Services',
       'Num_of_Medicare_Beneficiaries', 'Total_Medicare_Payment_Amt',
       'Average_Age_of_Beneficiaries', 'Medical_school_name',
       'Graduation_year', 'Primary_specialty', 'Organization_legal_name',
       'city_name', 'population_est', 'people_sq_mile', 'Years_in_Practice',
       'Full_Name', 'HYDROCODONE_day_count', 'TRAMADOL_day_count',
       'Percocet_day_count', 'oxycodone_day_count', 'fentanyl_day_count',
       'levothyroxine_day_count', 'potassium_day_count', 'metformin_day_count',
       'diltiazem_day_count', 'gabapentin_day_count',
       'HYDROCODONE_script_count', 'tramadol_script_count',
       'Percocet_script_count', 'oxycodone_script_count',
       'fentanyl_script_count', 'levothyroxine_script_count',
       'potassium_script_count', 'metformin_script_count',
       'diltiazem_script_count', 'gabapentin_script_